In [ ]:
pip install pandas requests beautifulsoup4 spacy

python -m spacy download xx_ent_wiki_sm

# python -m spacy download en_core_web_sm

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
import requests
from bs4 import BeautifulSoup

# Make a request to the webpage
url = 'https://nasional.kompas.com/read/2017/11/07/18102221/di-sidang-mk-peneliti-lipi-nilai-ahmadiyah-tak-bisa-dianggap-sesat?page=all'  # Replace with the actual URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Initialize an empty dictionary
scraped_data = {}

scraped_data['title'] = soup.find(class_='read__title').text
scraped_data['topic_subtitle'] = soup.select_one('.topicSubtitle ul li a').text
scraped_data['read_time'] = soup.select_one('.read__time')
scraped_data['writers'] = [h6.text for h6 in soup.select('.credit-title-name')]
scraped_data['read_content'] = '\n'.join(p.text for p in soup.select('.read__content p'))

In [4]:
# Convert the dictionary to a Pandas DataFrame
df = pd.DataFrame([scraped_data])

In [5]:
df.to_csv('scraped_data.csv', index=False)

# Extract Entity

In [6]:
import spacy

# Load the Indonesian model for spaCy
nlp = spacy.load('xx_ent_wiki_sm')  # This is the small Indonesian model

# Assuming 'read_content' is the variable containing the scraped content
doc = nlp(scraped_data['read_content'])

# Extract entities
entities = [(ent.text, ent.label_) for ent in doc.ents]

In [8]:
s = set()
for entity in entities:
    s.add(entity[0] + "|" + entity[1])

In [9]:
for i in s:
    print(i)

Penganut|LOC
Ahmadiyah|PER
JAI|MISC
SKB|ORG
Baca|PER
Undang-Undang Penodaan Agama|LOC
Undang-Undang Nomor|LOC
Upaya|LOC
Sanjungan|PER
SKB Tiga Menteri|ORG
LIPI|ORG
Jakarta Pusat|PER
Tazkirah|PER
pun|ORG
Ahmad Najib Burhani|PER
Ghulam Ahmad|PER
Keputusan Bersama|PER
RI|ORG
Islam|MISC
Selanjutnya|LOC
Qadian|PER
Perintah|MISC
Jemaah Ahmadiyah|PER
Masjid Mubarak|PER
NU|LOC
saya ingin menunjukkan|PER
Al Quran|PER
Apakah Ghulam Ahmad|PER
Nomor J.A.|PER
PNPS|ORG
Senin|LOC
Wakil Ketua Majelis Pustaka|LOC
Komnas HAM|MISC
yang jelas|PER
Baca|LOC
Pasal 1|MISC
hitungan hari|PER
Saya|PER
Nilai Negara|PER
Pemerintah Kota Depok|PER
NU|PER
Sebelumnya|LOC
Qadian|LOC
Berdasarkan pengalaman itu|MISC
Kompas.com/Alsadad Rudi Pelang|PER
PNPS/1965|LOC
Sidang MK|MISC
Informasi PP Muhammadiyah|PER
India|LOC
Baitullah di Mekkah|PER
yang justru|PER
Masjid Al Aqsa|PER
Jamaah Ahmadiyah|PER
Namun|PER
Depok|LOC
yang tidak benar|PER
Keputusan Menteri Kehakiman|PER
Nabi Muhammad|PER
Diskriminasi Berlapis|PER
penelitia

# Pagination

In [11]:
import urllib.parse

In [12]:
import requests
from bs4 import BeautifulSoup

# Define the base URL and search parameters
scraped_data = []

# Function to scrape a single page
def scrape_page(query, page):
    base_url = "https://search.kompas.com/search/"
    safe_string = urllib.parse.quote_plus(query)
    params = {
        "q": safe_string,
        "submit": "Submit",
        "gsc.tab": "0",
        "gsc.q": query.replace(' ', '%20'),
        "gsc.sort": "date",
        "gsc.page": page
    }

    print(params)
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, 'html.parser')

    print(soup.get_text)

    # Print the list of hrefs
    # print(hrefs)

# Process the first page
org = "Front Pembela Islam (FPI)"
scrape_page(org, 1)

{'q': 'Front+Pembela+Islam+%28FPI%29', 'submit': 'Submit', 'gsc.tab': '0', 'gsc.q': 'Front%20Pembela%20Islam%20(FPI)', 'gsc.sort': 'date', 'gsc.page': 1}
<bound method PageElement.get_text of <!DOCTYPE html>

<html lang="en">
<head>
<script>
        window.dataLayer = window.dataLayer || [];
        window.dataLayer.push({
            "content_category": "search",
        });
    </script>
<script>
    window.dataLayer = window.dataLayer || [];
    window.dataLayer.push([]);</script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-PT7PC4');</script>
<!-- End Google Tag Manager -->
<title>Berita Terkini Hari Ini, Kabar Akurat Terpercaya - Kompas.com</title>
<link href="//se

In [13]:
from bs4 import BeautifulSoup
import requests 

url = 'https://indeks.kompas.com/?site=all&date=2023-11-03&page=1'
req = requests.get(url)

# print(req.text)

soup = BeautifulSoup(req.text, 'lxml')

a = soup.find_all('a',{'class':'article__link'})

kumpulan_link = []
kumpulan_paragraf = []

for link in a: 
    kumpulan_link.append(link['href'])

for link in kumpulan_link:
    halaman = requests.get(link)
    soup_baru = BeautifulSoup(halaman.text,'lxml')
    paragraf = soup_baru.find_all('p')
    for kalimat in paragraf: 
        kumpulan_paragraf.append(kalimat.text)

with open('paragraf.txt', 'a') as f: 
    for paragraf in kumpulan_paragraf: 
        print('penulisan berhasil')
        f.writelines(paragraf + '\n')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

# From pages

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('organization.csv')

for index, row in df.iterrows():
    # Access each column in the row using row['column_name']
    print(row['organization_name'])

Al-Irsyad Al-Islamiyyah
Al Ittihadiyah
Alkhairaat
Al Washliyah
Darud Da'wah wal Irsyad (DDI)
Dewan Dakwah Islamiyah Indonesia (DDII)
Hidayatullah
Jamiat Kheir
Mathla'ul Anwar
Muhammadiyah (MU)
Nahdlatul Ulama (NU)
Nahdlatul Wathan (NW)
Persatuan Islam (Persis)
Persatuan Tarbiyah Islamiyah (PERTI)
Persatuan Umat Islam (PUI)
Rabithah Alawiyah
Wahdah Islamiyah
Ahlulbait Indonesia (ABI)
Ikatan Jamaah Ahlulbait Indonesia (Ijabi)
Dewan Masjid Indonesia (DMI)
Forum Umat Islam (FUI)
Ikatan Cendekiawan Muslim Indonesia (ICMI)
Ikatan Da'i Indonesia (Ikadi)
Lembaga Persahabatan Ormas Islam Indonesia (LPOI)
Lembaga Dakwah Islam Indonesia (LDII)
Majelis Intelektual dan Ulama Muda Indonesia (MIUMI)
Majelis Tafsir Al-Qur'an (MTA)
Majelis Ulama Indonesia (MUI)
Persatuan Islam Tionghoa Indonesia (PITI)
Syarikat Islam (SI)
Syarikat Islam Indonesia (SII)
Front Pembela Islam (FPI)
Gerakan Fajar Nusantara (Gafatar)
Hizbut Tahrir Indonesia (HTI)
Jamaah Ansharut Daulah (JAD)
Jamaah Ansharusy Syariah (JAS)
Ja